In [11]:
import os
from transformers import AutoModel, AutoTokenizer, BertModel, BertConfig
import torch.nn as nn
import re

# import gluonnlp
import numpy as np
from scipy import stats
import torch
import torch.nn as nn 
import logging
logging.getLogger().setLevel(logging.ERROR)

from tqdm.auto import tqdm

GPUIndex = str("0")

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= GPUIndex

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## DefRank

In [83]:
# Embedding = Embedding.to(device)
CandidateNum = 3
distance = nn.MSELoss(reduction='none')
# distance = nn.CosineEmbeddingLoss(reduction='none')
correct, total = 0, 0
WordDef_test_src, WordDef_test_tgt = [], []
# ModelName = 'bert-base-uncased'
# ModelName = 'bert-large-uncased'
# ModelName = 'roberta-base'
# ModelName = 'sentence-transformers/bert-base-nli-stsb-mean-tokens'
# ModelName = '/ckpt/def-bert/GlossBERT/'
# ModelName = '/ckpt/def-bert/save/mlm/bert/checkpoint-21605/'
# ModelName = '/ckpt/def-bert/save/mlm/sent-bert/checkpoint-2161/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-D[CLS]/'
# ModelName = '/ckpt/def-bert/save/D[CLS]-E[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-W[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-E[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-W[TGT]+W[CLS]-D[CLS]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-E[TGT]+W[CLS]-D[CLS]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-W[TGT]+D[CLS]-E[TGT]/'
ModelName = '/ckpt/def-bert/save/W[CLS]-E[TGT]+D[CLS]-E[TGT]/'
# ModelName = './save/WD+DE+WW+WE_best/'

Model = AutoModel.from_pretrained(ModelName)
# Model = BertModel(BertConfig())

Tokenizer = AutoTokenizer.from_pretrained(ModelName)
Model.eval()
print("Loaded . ")
Model = nn.DataParallel(Model).to(device)

# Defs = []
# for w in VocabPOSDefExamDict.keys():
#     for p in VocabPOSDefExamDict[w].keys():
#         Defs += list(VocabPOSDefExamDict[w][p].keys())
# print(len(Defs))

# FileObject = open("./data/DefRank_easy.txt", 'r', encoding='utf-8')
# FileObject = open("./data/DefRank_challenge.txt", 'r', encoding='utf-8')
FileObject = open("./data/DefRank_neo.txt", 'r', encoding='utf-8')

pbar = tqdm()
for line in FileObject:
    line = line.strip().split(' | ')
    Word = line[0]; Defs = line[1:]
    Word = Tokenizer(Word, add_special_tokens=True, padding=True, return_tensors='pt')
    Defs = Tokenizer(Defs, add_special_tokens=True, padding=True, return_tensors='pt')
    with torch.no_grad():
        WordEmb = Model(**Word)['last_hidden_state'][:,0,:] # CLS
        DefEmb = Model(**Defs)['last_hidden_state'][:,0,:]
        WordEmb = WordEmb.repeat(DefEmb.size(0),1)
        Distance = distance(WordEmb, DefEmb)
        Distance = torch.mean(Distance, dim=1, keepdim=True)
        
        _, prediction = torch.min(Distance, 0)
        correct += (prediction == 0).sum().item()
        total += prediction.size(0)
        print(np.round(correct*100/total, 2), end='\r')
    pbar.update(1)
FileObject.close()
pbar.close()

Loaded . 


0it [00:00, ?it/s]

In [ ]:
# 36.37
#1 40.76, 26.79 41.16
#1 45.75, 25.97, 44.06

In [ ]:
#2 40.74, 

In [ ]:
#5 41.85?

## SenseRank

In [85]:
import logging

distance = nn.MSELoss(reduction='none')
# distance = nn.CosineEmbeddingLoss(reduction='none')
correct, total = 0, 0
# ModelName = 'bert-base-uncased'
# ModelName = 'bert-large-uncased'
# ModelName = 'roberta-base'
# ModelName = 'sentence-transformers/bert-base-nli-stsb-mean-tokens'
# ModelName = '/ckpt/def-bert/GlossBERT/'
# ModelName = '/ckpt/def-bert/save/mlm/bert/checkpoint-21605/'
# ModelName = '/ckpt/def-bert/save/mlm/sent-bert/checkpoint-2161/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-D[CLS]/'
# ModelName = '/ckpt/def-bert/save/D[CLS]-E[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-W[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-E[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-W[TGT]+W[CLS]-D[CLS]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-E[TGT]+W[CLS]-D[CLS]/'
ModelName = '/ckpt/def-bert/save/W[CLS]-W[TGT]+D[CLS]-E[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-E[TGT]+D[CLS]-E[TGT]/'
# ModelName = './save/WD+DE+WW+WE_best/'

Model = AutoModel.from_pretrained(ModelName)
Tokenizer = AutoTokenizer.from_pretrained(ModelName)
Model.eval()
print("Loaded . ")
Model = nn.DataParallel(Model).to(device)

FileObject = open("./data/SenseRank_test.txt", 'r', encoding='utf-8')
Extractor = torch.tensor([0]).unsqueeze(1).expand(-1,10).to(device)

pbar = tqdm()
for line in FileObject:
    line = line.strip().split(' | ')
    Word = line[0]; Exam = line[1]; Defs = line[2:]
    Word = Tokenizer(Word, add_special_tokens=False, padding=True, return_tensors='pt')
    Defs = Tokenizer(Defs, add_special_tokens=True, padding=True, return_tensors='pt')
    Exam = Tokenizer(Exam, add_special_tokens=True, padding=True, return_tensors='pt')

    idx = (Exam["input_ids"] == Word["input_ids"][:,0].expand(1,Exam["input_ids"].size(1))).nonzero()#[0].unsqueeze(1).expand(-1,512))
    wlen = Word["input_ids"].size(1)
    with torch.no_grad():
        DefEmb = Model(**Defs)['last_hidden_state'][:,0,:]
        WordEmb = Model(**Exam)['last_hidden_state']
    if len(idx):
        idx = idx[0]
    else:
        pbar.update(1)
        continue
    WordEmb = torch.mean( WordEmb[idx[0]:idx[0]+1,idx[1]:idx[1]+wlen,:], dim=1)
    WordEmb = WordEmb.repeat(DefEmb.size(0),1)
    Distance = distance(WordEmb, DefEmb)
    Distance = torch.mean(Distance, dim=1, keepdim=True)

    _, prediction = torch.min(Distance, 0)
    correct += (prediction == 0).sum().item()
    total += prediction.size(0)
    print(np.round(correct*100/total, 2), end='\r')
#     break
    pbar.update(1)
FileObject.close()
pbar.close()

Loaded . 


0it [00:00, ?it/s]

In [51]:
59.37

80.8 31.19
